# Getting started with Python in DataONE

The DataONE python libraries are located in GitHub at [github.com/DataONEorg/d1_python](https://github.com/DataONEorg/d1_python) with documentation at [dataone-python.readthedocs.io](https://dataone-python.readthedocs.io/en/latest/).

## Installation

Installation under a virtual environment is recommended. 

Pip:
```
pip install -U dateparser
pip install -U dataone-common
pip install -U dataone-libclient
pip install -U dataone-scimeta
pip install -U dataone-util
```

Developer installation for editing the library source:
```
git clone https://github.com/DataONEorg/d1_python.git
cd d1_python
pip install -U dateparser
pip install -e lib_common/src
pip install -e lib_client/src
pip install -e utilities/src
pip install -e lib_scimeta/src
```

## Getting a client

The DataONE python clients provide an abstraction of the DataONE APIs enabling interaction with DataONE Member (MN) and Coordinating (CN) Nodes. Since the APIs of MNs and CNs differ, there are two basic types of client - one for MNs and one for CNs that derive from a common base. There are also different versions of the DataONE API in use by MNs, though these have mostly upgraded to version 2. CNs are all operating with the version 2 API.

### Getting a MN client

In [ ]:
import d1_client.mnclient_2_0

# Base URL of the KNB MN
base_url = "https://knb.ecoinformatics.org/knb/d1/mn"
client = d1_client.mnclient_2_0.MemberNodeClient_2_0(base_url)

# Call the getCapabilities API method
# https://dataone-architecture-documentation.readthedocs.io/en/latest/apis/MN_APIs.html#MNCore.getCapabilities
node = client.getCapabilities()

# Response is an instance of a Node document that can be accessed through its properties
# https://dataone-architecture-documentation.readthedocs.io/en/latest/apis/Types.html#Types.Node
print(f"{node.name:30} {node.baseURL}")

### Getting a CN client

In [ ]:
import d1_client.cnclient_2_0

# The production CN base URL:
base_url = "https://cn.dataone.org/cn"
client = d1_client.cnclient_2_0.CoordinatingNodeClient_2_0(base_url)

# Call the listNodes API method
# https://dataone-architecture-documentation.readthedocs.io/en/latest/apis/CN_APIs.html#CNCore.listNodes
nodes = client.listNodes()

# Response is a structure mirroring the API response message structure, 
# in this case a list of Node objects
# https://dataone-architecture-documentation.readthedocs.io/en/latest/apis/Types.html#Types.NodeList
for node in nodes.node:
    print(f"{node.name:30}\n  {node.baseURL}")


## Handling Errors

Errors arising from the APIs will generally result in a `DataONEException` which may include additional details about the error condition.

For example, attempting to get System Metadata for a non-existing object:

In [ ]:
# Try and retrieve non-existing system metadata
import d1_common.types.exceptions

identifier = "something_bogus"
try:
    sysmeta = client.getSystemMetadata(identifier)
except d1_common.types.exceptions.DataONEException as e:
    print(e)

## Response and request message structure

The python library parses and validates all XML messages returned from a node. They are deserialzed from XML to a python object (actually a PyXB wrapper) enabling access to the message properties.

Properties are named the same as their descriptions in the documentation, and values can generally be accessed directly through the property name. A couple of exceptions are [`identifier`](https://dataone-architecture-documentation.readthedocs.io/en/latest/apis/Types.html#Types.Identifier) and [`subject`](https://dataone-architecture-documentation.readthedocs.io/en/latest/apis/Types.html#Types.Subject), the values for which are accessed through their `value()` method.

Using the [`nodeList`]() response from the `CN.listNodes()` call above:

In [ ]:
nodes.__class__.__name__

A node list is a list of Node instances:

In [ ]:
node = nodes.node[0]
node.__class__.__name__

The `name`, `baseURL` and `identifier` properties of the node documents can be retrieved. Note that the identifier value must be accessed through the value() property getter:

In [ ]:
print(node.name)
print(node.baseURL)
print(node.identifier)
print(node.identifier.value())